In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from scipy.spatial.distance import cdist #for euclidean distance of consistency
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import uuid #to generate random id
import pickle #to save/load list of selected hens

#test equal variance
from scipy.stats import levene

#clustering
from sklearn.cluster import AgglomerativeClustering 

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#interpolate curves for clustering among birds with not exact same days tracked
#from scipy.interpolate import interp1d

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score, r2_score,\
mean_squared_error, mean_absolute_error, explained_variance_score#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
from scipy.stats import pearsonr, spearmanr 
import scipy.stats as stats

#PCA
from sklearn import decomposition

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns
import dexplot as dxp #for barplot
from mpl_toolkits.mplot3d import Axes3D

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, ts_visual, time_series_henColumn_tsRow, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
path_extracted_data_SNA = config.path_extracted_data_SNA
#create a director if not existing
if not os.path.exists(path_extracted_data_SNA):
    os.makedirs(path_extracted_data_SNA)
print(id_run)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [5]:
#focal birds info (one row per bird)
df_FB = pd.read_csv(os.path.join(path_extracted_data,id_run+'df_FOCALBIRDS.csv'), sep=';', parse_dates=['InitialStartDate'],
                     dayfirst=True) 
df_FB['percentage_of_gain_weight'] = df_FB.apply(lambda x: (x['weight 23-11-2020']-x['29-09 weight'])/x['29-09 weight']*100, axis=1)
print(df_FB.shape)
df_FB.head(3)

(212, 12)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,PenID,CLASS,R-Pen,InitialStartDate,29-09 weight,10-12 juin weight,weight 23-11-2020,weight 04-01-2021,weight 01-02-21,Treatment,percentage_of_gain_weight
0,hen_1,9,EPI,1.0,2020-06-10,1134.0,57.0,1696.5,1787.8,1800.9,OFH,49.603175
1,hen_10,11,LEXP,1.0,2020-06-10,1070.0,70.4,1488.3,1628.4,1602.1,OFH,39.093458
2,hen_100,5,EPI,3.0,2020-06-10,1182.0,80.5,1642.5,NaN,NaN,OFH,38.959391


In [6]:
#cleaned data of the tracking system movements
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', 
                 parse_dates=['Timestamp', 'date'], dayfirst=True) 
df['hour'] = df['Timestamp'].map(lambda x: x.hour)
df['time'] = df['Timestamp'].map(lambda x: dt.datetime.time(x-dt.timedelta(seconds=x.second)))
df = df[df['HenID'].isin(li_selected_hens)] 
df.drop('duration', axis=1, inplace=True)
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'max_date_adaptability' is not defined

In [ ]:
#daily variables (one row per (henID, date))
df_daily = pd.read_csv(os.path.join(path_extracted_data, 'daily_ALL_Variable_Tranformed.csv'), sep=';',
                     parse_dates=['level','FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone',
                              'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone', 
                              'Nestbox_time_of_first_staid_longer_than900sec',
                              'duration_last-firsttransition_mn'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by dates
df_daily = df_daily[df_daily['level']<=max_date_adaptability] 
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]

#filter the selected hens
df_daily = df_daily[df_daily['HenID'].isin(li_selected_hens)] 

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
print(df_daily.shape)
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)

#remove the days where the night had movement recorded but not the days
print(df_daily.shape)
#display(df_daily[df_daily.isna().any(axis=1)])
df_daily = df_daily[~df_daily['verification_daily_total_duration'].isnull()]
print(df_daily.shape)

df_daily['dur_values_normalized'].replace('[nan, nan, nan, nan, nan]','[np.nan,np.nan,np.nan,np.nan,np.nan]', inplace=True)
df_daily['dur_values_normalized'] = df_daily['dur_values_normalized'].map(lambda x: eval(x))
df_daily['duration_last-firsttransition_mn'] = df_daily['duration_last-firsttransition_mn'].astype(float)
print(df_daily.shape)
df_daily.head(3)

In [ ]:
#download the MLPs vectors during day only! as we compare on same day, we can restrict to the exact daily calendar!
dico_pen_bin_zone_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_zone_level_h_DAILYLEVEL.pkl'), 'rb'))
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_level_h_DAILYLEVEL.pkl'), 'rb'))
#dico of hen as keys and pen as values
df_daily['PenID'] = df_daily['PenID'].map(lambda x: 'pen'+(str(int(x))))
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))

# Compute the distance across pairs of MLPS

In [9]:
#compute the CHI2DISTANCE & DTW for each bird with all other birds
li_df = []
df.sort_values(['Timestamp'], inplace=True)
#take (subset of) dates and hens that are available (i.e. for which we comptue the variable on)
#df_daily_h = df_daily[(df_daily['DOA']>=150)&(df_daily['DOA']<=380)].copy()
li_date = set(df_daily_h['level'].tolist())
for d in tqdm.tqdm(sorted(list(li_date))):
    li_hen_d = df_daily_h[df_daily_h['level']==d]['HenID'].unique()
    #its a symmetric measure, so we only do each combination once
    for i,h1 in enumerate(li_hen_d):

        #lets compute for hen h1 its difference with all other birds
        for h2 in li_hen_d[i+1:]:
            
            ############ Overall aviary usage similarity across days
            l1_chi2 = df_daily_h[(df_daily_h['HenID']==h1)&(df_daily_h['level']==d)]['dur_values_normalized'].values[0]
            l2_chi2 = df_daily_h[(df_daily_h['HenID']==h2)&(df_daily_h['level']==d)]['dur_values_normalized'].values[0]
            dico_ = {'date':d, 'henId':h1, 'henID2':h2, 'chi2distance':chi2_distance(l1_chi2,l2_chi2)}
            #'dur_values_normalized_henId':l1_chi2,'dur_values_normalized_henID2':l2_chi2,

            ############# Specifics' zones usage similarity across days
            for nbr_binmn in li_binmn:
                #ALL zone together with chi2distance
                l1_chi2 = dico_pen_bin_level_h[dico_h_p[h1]][nbr_binmn][d][h1]
                l2_chi2 = dico_pen_bin_level_h[dico_h_p[h2]][nbr_binmn][d][h2]
                dico_['nbr_obs_chi2all_'+str(nbr_binmn)] = len(l1_chi2)
                li_chi2 = [chi2_distance(l1_chi2[i],l2_chi2[i]) for i in range(0,len(l1_chi2))]
                dico_['li_chi2_'+str(nbr_binmn)] = li_chi2
                dico_['chi2distance_ALL_'+str(nbr_binmn)] = np.mean(li_chi2)
                
                #per zone with DTW
                for ZONE in li_zone:
                    dtw_value = np.nan
                    try:
                        l1_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h1]][nbr_binmn][ZONE][d][h1], dtype=np.double)
                        l2_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h2]][nbr_binmn][ZONE][d][h2], dtype=np.double)
                        #compute distance measure
                        #psi: #of start and end points of a sequence can be ignored if this would lead to a lower distance
                        dtw_value, paths = dtw.warping_paths(l1_dtw, l2_dtw, window=dico_window[nbr_binmn], psi=0, penalty=penalty)
                    except Exception as e:
                        print(e)
                        sys.exit()
                        pass #dont do anything
                    dico_['DTW_'+str(nbr_binmn)+'_'+ZONE] = round(dtw_value,1)  
                    dico_['nbr_obs_l1_'+str(nbr_binmn)+'_'+ZONE] = len(l1_dtw)
                    dico_['nbr_obs_l2_'+str(nbr_binmn)+'_'+ZONE] = len(l2_dtw)
            li_df.append(dico_)

#turn inter and intra distances into a dataframe                
df_inter = pd.DataFrame(li_df)
#df_inter = df_inter[~df_inter['chi2distance'].isnull()]
df_inter = df_inter.sort_values('DOA', ascending=True)
df_inter['hen_pair'] = df_inter.apply(lambda x: '-'.join(sorted([x['henId'],x['henID2']])), axis=1)
df_inter['DTW_15_all'] = df_inter[['DTW_15_1_Zone', 'DTW_15_2_Zone', 'DTW_15_3_Zone','DTW_15_4_Zone','DTW_15_5_Zone']].sum(axis=1)
li_col = list(df_inter.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
#df_inter.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
#df_inter.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
df_inter.to_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceBetweenHenSim_DAILYLEVEL.csv'), sep=';', index=False)
print(df_inter.shape)
display(df_inter.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|███████████████████████████████████████████████████████████████████████████████| 52/52 [6:25:33<00:00, 444.87s/it]


['li_chi2_5', 'li_chi2_10', 'li_chi2_15', 'li_chi2_20', 'li_chi2_30']
(397789, 100)


,level1,level2,henID1,henID2,chi2distance,DOA,WOA,nbr_obs_chi2all_5,li_chi2_5,chi2distance_ALL_5,...,nbr_obs_l2_30_5_Zone,DTW_30_4_Zone,nbr_obs_l1_30_4_Zone,nbr_obs_l2_30_4_Zone,DTW_30_1_Zone,nbr_obs_l1_30_1_Zone,nbr_obs_l2_30_1_Zone,weeks_in_laying_barn,hen_pair,DTW_15_all
0,2020-09-30,2020-09-30,hen_1,hen_10,0.233889,119,17,97,"[0.9999999999989999, 0.9999999999989999, 0.999...",0.676595,...,17,20.8,17,17,0.0,17,17,1,hen_1-hen_10,154.0
5686,2020-09-30,2020-09-30,hen_2,hen_47,0.341557,119,17,97,"[0.9999999999989999, 0.9999999999989999, 0.023...",0.687179,...,17,23.0,17,17,0.0,17,17,1,hen_2-hen_47,158.6
5685,2020-09-30,2020-09-30,hen_2,hen_46,0.110694,119,17,97,"[0.0, 0.0948905109484006, 0.5267175572512697, ...",0.647620,...,17,23.0,17,17,0.0,17,17,1,hen_2-hen_46,152.9


# Compute Network

In [8]:
path_extracted_data_visual_adap = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability')
df_inter = pd.read_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim_DAILYLEVEL.csv'), sep=';')
display(df_inter.head(3))
df_inter.shape

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyboardInterrupt: 

In [ ]:
plt.hist(df_inter['DTW_15_all']);

In [ ]:
plt.hist(df_inter['DTW_15_1_Zone']);

In [ ]:
plt.hist(df_inter['DTW_15_2_Zone']);

In [ ]:
plt.hist(df_inter['DTW_15_3_Zone']);

In [ ]:
plt.hist(df_inter['DTW_15_4_Zone']);

In [ ]:
plt.hist(df_inter['DTW_15_5_Zone']);

In [ ]:
dico_distance_mindist = {'DTW_15_all':, 
                         'DTW_15_1_Zone':, 
                         'DTW_15_2_Zone':, 
                         'DTW_15_3_Zone':,
                         'DTW_15_4_Zone':,
                         'DTW_15_5_Zone':}
li_df = []
for d in tqdm.tqdm(sorted(list(li_date))):
    
    df_graph = df_inter[df_inter['date']==d].copy()
    
    #one graph per distance
    for D,min_dist in dico_distance_mindist.items():
        #initialise graph
        G = nx.Graph()
        #assign graph attributes (i.e. similarity computation)
        G.graph['similarity computation']= D

        #add nodes (s: with attributes if needed)
        li_hen_d = df_daily_h[df_daily_h['level']==d]['HenID'].unique()
        for h in li_hen_d:
            #s = max(df_movement[df_movement['HenID']==h]['severity'].tolist())
            G.add_node(h) #attributes:, max_severity=int(round(s,0)))

        #add edges
        li_edges = [] 
        #we use a symmetric measure, so we only do each combination once
        for i,h1 in enumerate(li_hen_d):
            #lets compute for hen h1 its difference with all other birds
            for h2 in li_hen_d[i+1:]:
                dist = df_graph[df_graph['hen_pair']=='-'.join(sorted([x['h1'],x['h2']]))][D].values
                pprint(dist)
                #link if dist < min_dist
                if dist<min_dist:
                    li_edges.append((h1, h2, {'similarity':int(round(dist,2))}))     
        G.add_edges_from(li_edges)

        #save G (networkX graph) in GEXF format for gephi
        nx.write_gexf(G, os.path.join(path_extracted_data_SNA,D+str(d).split(' ')[0]+'.gexf'))
        
        #centrality measures
        #returns a Dictionary of nodes with degree centrality as the value
        #The degree centrality values are normalized by dividing by the maximum possible degree in a simple graph n-1 where n is 
        #the number of nodes in G.
        dico_v_dg = degree_centrality(G)
        #Compute the shortest-path betweenness centrality for nodes. see doc for additional info
        #return a Dictionary of nodes with betweenness centrality as the value.
        dico_v_bw = betweenness_centrality(G, seed=0)
        #Compute closeness centrality for nodesee doc for additional info
        #return a Dictionary of nodes with closeness centrality as the value
        dico_v_cl = closeness_centrality(G)
        #into a datfarme
        df_ = pd.DataFrame(list(dico_v_cl.items()),columns = ['HenID','closeness'])
        df_['betweeness'] = df_['HenID'].map(lambda x: dico_v_bw[x])
        df_['degree'] = df_['HenID'].map(lambda x: dico_v_dg[x])
        df_['date'] = d  
        df_['distance_measure'] = D
        li_df.append(df_)
        sys.exit()
df_SNA = pd.concat(li_df,axis=1)
print(df_SNA.shape)
df_SNA.head(3)